# Relaxation

The basic idea is to get rid of the kinks in the value function by relaxing the problem.
In particular, we relax it to a convex optimization problem.

For two variables, the constraints $0 \leq \max{x_1, x_2} \leq 1$ describe a size 1 square in the first quadrant.
The idea is, to instead take something like a unit ball centered at 0.5, 0.5, which is strictly larger.
In particular, we have $(x_1-0.5)^2 + (x_2-0.5)^2<=1$ as the smallest circle that still includes the old parameter space.

Adding this constraint turns the linear program into a convex program, since one of the constraints is no longer linear.

In [ ]:
%load_ext autoreload
%autoreload 2

from functools import partial

import jax.numpy as jnp
import numpy as np
import optimagic as om
import plotly.graph_objects as go
from jax import grad

from lp_relax.funcs.lp_relax import generate_sphere_constraint

In [ ]:
def _linear_objective(params, slope, slope_y=0.5):
    return slope * params[0] + slope_y * params[1]


bounds = om.Bounds(lower=np.array([0, 0]), upper=np.array([1, 1]))

lbfgsb_res = om.minimize(
    fun=_linear_objective,
    params=[0.5, 0.5],
    algorithm="scipy_lbfgsb",
    bounds=bounds,
    fun_kwargs={"slope": 1},
)

linear = partial(
    om.minimize,
    fun=_linear_objective,
    params=np.ones(2) * 0.5,
    algorithm="scipy_lbfgsb",
    bounds=bounds,
)

In [ ]:
convex_problems = {
    k: partial(
        om.minimize,
        fun=_linear_objective,
        params=np.ones(2) * 0.5,
        algorithm="scipy_cobyla",
        constraints=generate_sphere_constraint(num_dims=2, k=k),
    )
    for k in [2, 4, 10]
}

In [ ]:
num_points = 1000
grid = np.linspace(-1, 1, num_points)

funs = {
    problem: np.zeros(num_points)
    for problem in ["linear", *list(convex_problems.keys())]
}

solutions = {
    problem: np.zeros((num_points, 2))
    for problem in ["linear", *list(convex_problems.keys())]
}

In [ ]:
for i, val in enumerate(grid):
    _res_linear = linear(fun_kwargs={"slope": val})
    funs["linear"][i] = _res_linear.fun
    solutions["linear"][i] = _res_linear.params

    for k, _res_convex in convex_problems.items():
        _res_convex = _res_convex(fun_kwargs={"slope": val})
        funs[k][i] = _res_convex.fun
        solutions[k][i] = _res_convex.params

In [ ]:
fig = go.Figure()

# Plot fun_linear and fun_convex against grid

fig.add_trace(
    go.Scatter(
        x=grid,
        y=funs["linear"],
        mode="lines",
        name="",
        legendgroup="linear",
        legendgrouptitle={"text": "Linear Constraint"},
    )
)

for k in convex_problems:
    fig.add_trace(
        go.Scatter(
            x=grid,
            y=funs[k],
            mode="lines",
            legendgroup="convex",
            name=f"k = {k}",
            legendgrouptitle={"text": "Convex Constraints"},
        )
    )

# Add note with number of grid points
fig.add_annotation(
    x=0.75,
    y=-1,
    text=f"Number of grid points: {num_points}",
    showarrow=False,
    align="center",
)

# Titles
fig.update_layout(
    title="Value Function by Constraint Set",
    xaxis_title="Slope of x1 (c1)",
    yaxis_title="Minimized Objective",
)

fig.show()

In [ ]:
# Construct cone constraints
a = np.array([1, 2, 3])

# The functions are
#
# -s x1 + x2^2 <= ub
# -s x1 + (x2-1)^2 <= ub
# x1^2 + -s x2 <= ub
# (x1-1)^2 + -s x2 <= ub
#
s = 1
ub = 10

slopes = np.array([[s, s, -s, -s, 1, 1, 1, 1], [1, 1, 1, 1, s, s, -s, -s]]).T

centers = np.array([[1, 1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 1, 1, 0, 0]]).T

exponents = np.array([[1, 1, 1, 1, 2, 2, 2, 2], [2, 2, 2, 2, 1, 1, 1, 1]]).T

cone_constraint_funcs = [
    lambda x, i=i: np.sum(slopes[i] * (x - centers[i]) ** exponents[i])
    for i in range(8)
]

cone_constraints = [
    om.NonlinearConstraint(
        func=cone_constraint_funcs[i],
        upper_bound=ub,
        derivative=grad(cone_constraint_funcs[i]),
    )
    for i in range(8)
]

In [ ]:
# Plot all points that satisfy all range(8) constraints
points = np.array(
    [[x1, x2] for x1 in np.linspace(-4, 4, 200) for x2 in np.linspace(-4, 4, 200)]
)

satisfied = np.array(
    [
        np.all(
            [
                constraint.func(point) <= constraint.upper_bound
                for constraint in cone_constraints
            ]
        )
        for point in points
    ]
)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=points[satisfied, 0],
        y=points[satisfied, 1],
        mode="markers",
        name="Satisfied",
    )
)

fig.add_trace(
    go.Scatter(
        x=points[~satisfied, 0],
        y=points[~satisfied, 1],
        mode="markers",
        name="Not Satisfied",
    )
)

fig.update_layout(
    title="Points Satisfying All Cone Constraints",
    xaxis_title="x1",
    yaxis_title="x2",
)

In [ ]:
constran_funcs = [constraint.func for constraint in cone_constraints]

[func([-0.1, 0.5]) <= ub for func in constran_funcs]

In [ ]:
# Solve the problem with cone constraints over the grid and plot solution
cone_problem = partial(
    om.minimize,
    fun=_linear_objective,
    params=np.ones(2) * 0.33,
    constraints=cone_constraints,
)

funs["cone"] = np.zeros(num_points)
solutions["cone"] = np.zeros((num_points, 2))

for i, val in enumerate(grid):
    _res_cone = cone_problem(fun_kwargs={"slope": val}, algorithm="scipy_cobyla")
    funs["cone"][i] = _res_cone.fun
    solutions["cone"][i] = _res_cone.params

fig = go.Figure()

# Plot fun_linear and fun_convex against grid

fig.add_trace(
    go.Scatter(
        x=grid,
        y=funs["linear"],
        mode="lines",
        name="",
        legendgroup="linear",
        legendgrouptitle={"text": "Linear Constraint"},
    )
)

for k in convex_problems:
    fig.add_trace(
        go.Scatter(
            x=grid,
            y=funs[k],
            mode="lines",
            legendgroup="convex",
            name=f"k = {k}",
            legendgrouptitle={"text": "Convex Constraints"},
        )
    )

fig.add_trace(
    go.Scatter(
        x=grid,
        y=funs["cone"],
        mode="lines",
        name="",
        legendgroup="cone",
        legendgrouptitle={"text": "Cone Constraints"},
    )
)

# Add note with number of grid points
fig.add_annotation(
    x=0.75,
    y=-1,
    text=f"Number of grid points: {num_points}",
    showarrow=False,
    align="center",
)

# Titles
fig.update_layout(
    title="Value Function by Constraint Set",
    xaxis_title="Slope of x1 (c1)",
    yaxis_title="Minimized Objective",
)

fig.show()

In [ ]:
res = cone_problem(fun_kwargs={"slope": 0.5}, algorithm="scipy_trust_constr")
om.criterion_plot(res, max_evaluations=1000)

In [ ]:
cone_problem(fun_kwargs={"slope": 0.5}, algorithm="ipopt")

In [ ]:
rho = -0.2

box_constraints_with_noise_upper_funcs = [
    lambda x, i=i: x[i] + rho * jnp.linalg.norm(x) for i in range(2)
]

box_constraints_with_noise_lower_funcs = [
    lambda x, i=i: -x[i] + rho * jnp.linalg.norm(x) for i in range(2)
]

box_constraints_with_noise_upper = [
    om.NonlinearConstraint(
        func=box_constraints_with_noise_upper_funcs[i],
        upper_bound=1,
        lower_bound=-10000000,
        derivative=grad(box_constraints_with_noise_upper_funcs[i]),
    )
    for i in range(2)
]
box_constraints_with_noise_lower = [
    om.NonlinearConstraint(
        func=box_constraints_with_noise_lower_funcs[i],
        upper_bound=0,
        lower_bound=-10000000,
        derivative=grad(box_constraints_with_noise_lower_funcs[i]),
    )
    for i in range(2)
]

box_constraints_with_noise = (
    box_constraints_with_noise_upper + box_constraints_with_noise_lower
)


om.minimize(
    fun=_linear_objective,
    params=np.ones(2) * 0.5,
    algorithm="ipopt",
    fun_kwargs={"slope": -0.5},
    constraints=box_constraints_with_noise,
)

In [ ]:
opt_box_with_noise = partial(
    om.minimize,
    fun=_linear_objective,
    params=np.ones(2) * 0.5,
    algorithm="ipopt",
    constraints=box_constraints_with_noise,
)

funs["box_with_noise"] = np.zeros(num_points)

for i, val in enumerate(grid):
    _res_box_with_noise = opt_box_with_noise(fun_kwargs={"slope": val})
    funs["box_with_noise"][i] = _res_box_with_noise.fun

fig = go.Figure()

# Plot fun_linear and fun_convex against grid

fig.add_trace(
    go.Scatter(
        x=grid,
        y=funs["linear"],
        mode="lines",
        name="",
        legendgroup="linear",
        legendgrouptitle={"text": "Linear Constraint"},
    )
)


fig.add_trace(
    go.Scatter(
        x=grid,
        y=funs["box_with_noise"],
        mode="lines",
        name="",
        legendgroup="box_with_noise",
        legendgrouptitle={"text": "Box Constraints with Noise"},
    )
)

# Add note with number of grid points
fig.add_annotation(
    x=0.75,
    y=-1,
    text=f"Number of grid points: {num_points}",
    showarrow=False,
    align="center",
)

# Titles
fig.update_layout(
    title="Value Function by Constraint Set",
    xaxis_title="Slope of x1 (c1)",
    yaxis_title="Minimized Objective",
)

fig.show()